<a href="https://colab.research.google.com/github/poojatambe/ONNXRuntime-Inference/blob/main/ONNX_Runtime_inference_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install onnxruntime
# !pip install skl2onnx
!pip install onnxcustom

In [30]:
import numpy
from onnxruntime import InferenceSession
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from skl2onnx import to_onnx
from onnx.tools.net_drawer import GetPydotGraph, GetOpNodeProducer
import matplotlib.pyplot as plt 
import os
from onnxcustom.utils import measure_time

# Data preparation

In [17]:
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)


## Fit the model

In [18]:
reg = GradientBoostingRegressor(random_state=1, n_estimators=5)
reg.fit(X_train, y_train)

GradientBoostingRegressor(n_estimators=5, random_state=1)

## Convert to onnx

In [25]:
model_onnx = to_onnx(reg, X_train.astype(numpy.float32))
with open("iris.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

# Onnx runtime inference

In [46]:
sess = InferenceSession("iris.onnx",
                        providers=['CPUExecutionProvider'])
input_name = sess.get_inputs()[0].name
pred_ort = sess.run(None, {input_name: X_test.astype(numpy.float32)})[0]
print("ort prediction",pred_ort[:5])


ort prediction [[1.021089 ]
 [1.021089 ]
 [1.4312539]
 [1.021089 ]
 [0.6115789]]
1.021089


## Sklearn prediction

In [27]:
pred_skl = reg.predict(X_test.astype(numpy.float32))
print("sklearn prediction", pred_skl[:5])


sklearn prediction [1.02108893 1.02108893 1.43125398 1.02108893 0.61157893]


## Difference between predictions

In [50]:
diff=0
for i in range(5):
  diff+=(pred_skl[i]-pred_ort[i][0])
print("Error in prediction:", diff/5)  

Error in prediction: 5.062248664700064e-09


## Measure time

In [36]:
# sklearn
context = {"reg": reg, 'X': X_test[:5]}
mt = measure_time(
    "reg.predict(X)", context, div_by_number=True)
mt['size'] = context['X'].shape[0]
mt['mean_obs'] = mt['average'] / mt['size']
print("sklearn prediction time for 5 test samples",mt['mean_obs'])

#onnxruntime
context = {"sess": sess, 'X': X_test[:5].astype(numpy.float32)}
mt2 = measure_time(
    "sess.run(None, {'X': X})[0]", context, div_by_number=True)
mt['ort'] = mt2['average'] / mt['size']
print("Onnx runtime inference time for 5 test samples",mt['ort'])


# Difference 
print("Onnx runtime is faster than normal prediction", (mt['mean_obs']-mt['ort']))

sklearn prediction time for 5 test samples 5.29322083995794e-05
Onnx runtime inference time for 5 test samples 3.051122000033501e-06
onnx runtime is faster than normal prediction 4.9881086399545896e-05
